

# 개요

변수 비교를 위한 모델


# 1. 파일 업로드 및 기본 세팅

In [ ]:

# AIDU Library Import
from aicentro.session import Session
from aicentro.framework.keras import Keras as AiduFrm
aidu_session = Session(verify=False)
aidu_framework = AiduFrm(session=aidu_session)

# 데이터 파일 경로
data_path = aidu_framework.config.data_dir

import subprocess
import sys

import warnings
# 불필요한 경고 출력을 방지합니다.
warnings.filterwarnings('ignore')


#필요한 모델 다운로드
subprocess.call([sys.executable,"-m","pip","install","xgboost"])
subprocess.call([sys.executable,"-m","pip","install","seaborn"])
subprocess.call([sys.executable,"-m","pip","install","lightgbm"])
subprocess.call([sys.executable,"-m","pip","install","catboost"])




## 러닝 모델 선택

In [ ]:
#러닝을 위한 모델 선택
model_select = {
    
    'randomforest' : 0,
    'lightgbm' : 0,
    'catboost' : 1,
    'xgboost' : 0,
    'softvoting' : 0  

}

## 모델 결과 출력 디렉토리

In [ ]:

#acc 결과 저장 디렉토리
acc_data_dir = data_path + "/fit/accuracy/acc_data.csv"


# 데이터 불러오기 및 기본 처리


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd


#파일 명
#파일 명

BEFORE_FILE_NAME = 'all'
AFTER_FILE_NAME = 'after'


#전처리 후 train 파일명
train_before_file_name = data_path + "/pre_data/train_preprocessing_"+BEFORE_FILE_NAME+".csv"

#전처리 후 train 파일명
train_after_file_name = data_path + "/pre_data/train_preprocessing_"+AFTER_FILE_NAME+".csv"

#featureimportance 모델 첫번쨰 파일
before_importance_dir = data_path + "/fit/feature_importance/model_Feature_Importance_"+BEFORE_FILE_NAME+".csv"

#featureimportance 모델 두번째 파일
after_importance_dir = data_path + "/fit/feature_importance/model_Feature_Importance_"+AFTER_FILE_NAME+".csv"


#featureimportance 비교파일
compare_importance_dir = data_path + "/fit/feature_importance/model_Feature_Importance_age.csv"

train_df = pd.read_csv(train_after_file_name)



In [ ]:
#학습하기위한 train과 validation 나눔
# 일반 데이터
# x_train, x_valid, y_train, y_valid = train_test_split(train_df.drop(['label_payment_yn'],axis=1), train_df['label_payment_yn'], test_size =0.05,random_state=21)
x_train, x_valid, y_train, y_valid = train_test_split(train_df.drop(['label_payment_yn'],axis=1), train_df['label_payment_yn'],random_state=21)


### 모델 학습 후 결과 시각화

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

my_predictions = {}

colors = ['r', 'c', 'm', 'y', 'k', 'khaki', 'teal', 'orchid', 'sandybrown',
          'greenyellow', 'dodgerblue', 'deepskyblue', 'rosybrown', 'firebrick',
          'deeppink', 'crimson', 'salmon', 'darkred', 'olivedrab', 'olive', 
          'forestgreen', 'royalblue', 'indigo', 'navy', 'mediumpurple', 'chocolate',
          'gold', 'darkorange', 'seagreen', 'turquoise', 'steelblue', 'slategray', 
          'peru', 'midnightblue', 'slateblue', 'dimgray', 'cadetblue', 'tomato'
         ]


#acc를 구해서 시각화해줌
def acc_eval(name_, pred, actual):
    global predictions
    global colors

    acc = (pred==actual).mean()
    my_predictions[name_] = acc


    y_value = sorted(my_predictions.items(), key=lambda x: x[1], reverse=False)  # 정확도 내림차순으로 sort
    
    df = pd.DataFrame(y_value, columns=['model', 'acc'])
    print(df)
    min_ = df['acc'].min() -1
    max_ = 1.2
    
    length = len(df)
    
    plt.figure(figsize=(10, length))
    ax = plt.subplot()
    ax.set_yticks(np.arange(len(df)))
    ax.set_yticklabels(df['model'], fontsize=15)
    bars = ax.barh(np.arange(len(df)), df['acc'])
    
    for i, v in enumerate(df['acc']):
        idx = np.random.choice(len(colors))
        bars[i].set_color(colors[idx])
        ax.text(v+0.1, i, str(round(v, 3)), color='k', fontsize=15, fontweight='bold')
        
    plt.title('Accuracy', fontsize=18)
    plt.xlim(min_,max_)
    
    plt.show()
    
#acc를 구해서 시각화해줌(cross_val_score 사용하는 경우)
def acc_eval_add(name_, acc):
    global predictions
    global colors

    my_predictions[name_] = acc


    y_value = sorted(my_predictions.items(), key=lambda x: x[1], reverse=False)  # 정확도 내림차순으로 sort
    
    df = pd.DataFrame(y_value, columns=['model', 'acc'])
    print(df)
    min_ = df['acc'].min() -1
    max_ = 1.2
    
    length = len(df)
    
    plt.figure(figsize=(10, length))
    ax = plt.subplot()
    ax.set_yticks(np.arange(len(df)))
    ax.set_yticklabels(df['model'], fontsize=15)
    bars = ax.barh(np.arange(len(df)), df['acc'])
    
    for i, v in enumerate(df['acc']):
        idx = np.random.choice(len(colors))
        bars[i].set_color(colors[idx])
        ax.text(v+0.1, i, str(round(v, 3)), color='k', fontsize=15, fontweight='bold')
        
    plt.title('Accuracy', fontsize=18)
    plt.xlim(min_,max_)
    
    plt.show()
    
    df.to_csv(acc_data_dir,index=False)
    

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
import time
from sklearn.model_selection import KFold

# 3. 모델학습 (랜덤 포레스트, XGboost, Light GBM, CATboost )

### 3.2. XGBoost 

In [ ]:
from xgboost import XGBClassifier

def xgboost_model_fit (x_train_xg, x_valid_xg, y_train_xg, y_valid_xg, model_name='XGB'):
    '''
    Cat boost 모델 학습을 위한 함수
    
    param
    x_train_cat: x train data
    x_valid_cat: x valid data
    y_train_cat: y train data
    y_valid_cat: y valid data
    model_name: 모델 학습 후 그래프 출력을 위한 텍스트
    '''
     
#     # General Parameter
#     booster='gbtree' # 트리,회귀(gblinear) 트리가 항상 더 좋은 성능을 내기 때문에 수정할 필요없다고한다.
    
#     silent=True  # running message출력안한다. 모델이 적합되는 과정을 이해하기위해선 False으로한다.
    
#     min_child_weight=10   # 값이 높아지면 under-fitting 되는 경우가 있다. CV를 통해 튜닝되어야 한다.
    
#     max_depth=8     # 트리의 최대 깊이를 정의함. 루트에서 가장 긴 노드의 거리. 8이면 중요변수에서 결론까지 변수가 9개거친다.
#                     # Typical Value는 3-10. 
    
#     gamma =0    # 노드가 split 되기 위한 loss function의 값이 감소하는 최소값을 정의한다. gamma 값이 높아질 수록 알고리즘은 보수적으로 변하고, loss function의 정의
#                 #에 따라 적정값이 달라지기때문에 반드시 튜닝.
    
#     nthread =4    # XGBoost를 실행하기 위한 병렬처리(쓰레드) 갯수. 'n_jobs' 를 사용해라.
    
#     colsample_bytree=0.8   # 트리를 생성할때 훈련 데이터에서 변수를 샘플링해주는 비율. 보통0.6~0.9
    
#     colsample_bylevel=0.9  # 트리의 레벨별로 훈련 데이터의  변수를 샘플링해주는 비율. 보통0.6~0.9
    
#     n_estimators =(int)   #부스트트리의 양 트리의 갯수. 
    
#     objective = 'reg:linear','binary:logistic','multi:softmax',
#                 'multi:softprob'  # 4가지 존재.
#             # 회귀 경우 'reg', binary분류의 경우 'binary',
#             # 다중분류경우 'multi'- 분류된 class를 return하는 경우 'softmax'
#             # 각 class에 속할 확률을 return하는 경우 'softprob'
    
#     random_state =  # random number seed.
#                     # seed 와 동일.

            
     #모델 선언
    xgb_model = XGBClassifier(n_jobs=-1)
    
    
#     #1008일 파라미터 최적화 결과
#     xgb_model = XGBClassifier(n_jobs=-1,
#           max_depth = 6,
#           learning_rate= 0.23,
#           n_estimators= 299,
#           gamma=1.0,
#           min_child_weight= 3.2,
#           max_delta_step=0.09,
#           subsample= 0.7,
#           colsample_bytree= 0.58  
#             )
    
    start=time.time()

    eval_set = [(x_valid_xg, y_valid_xg)]
    
    #학습
    #logloss가 10회 계선되지 않으면 중단하도록 early stopping 설정 n_estimators=100에서는 얼리스탑하지 않음
    xgb_model.fit(x_train_xg, y_train_xg, early_stopping_rounds=10, eval_metric="logloss", eval_set=eval_set, verbose=2)

    (time.time()-start)/60
    
    pred= xgb_model.predict(x_valid_xg)
    
    acc_eval(model_name, pred,y_valid) #기본
    
    #모델 및 예측값 return
    return xgb_model, pred

In [ ]:
#xgboost 모델 선택시

if model_select['xgboost'] == 1:
    #모델 학습
    xgboost_model, xgboost_pred = xgboost_model_fit (x_train, x_valid, y_train, y_valid, 'xgboost' )
    #모델 save

    


### 3.3 CATBoost

In [ ]:
from catboost import CatBoostClassifier

def catboost_model_fit (x_train_cat, x_valid_cat, y_train_cat, y_valid_cat, model_name='CAT'):
    '''
    Cat boost 모델 학습을 위한 함수
    
    param
    x_train_cat: x train data
    x_valid_cat: x valid data
    y_train_cat: y train data
    y_valid_cat: y valid data
    model_name: 모델 학습 후 그래프 출력을 위한 텍스트
    '''
    #모델 선언
    cat_model= CatBoostClassifier()
    start=time.time()
    
    #학습
    cat_model.fit(x_train_cat, y_train_cat, verbose=0 )
    
    #예측 및 결과 확인
    pred = cat_model.predict(x_valid_cat)
    acc_eval(model_name ,pred, y_valid_cat)
    
    #모델 및 예측값 return
    return cat_model, pred




In [ ]:
before_col = []
after_col = []

if model_select['catboost'] == 1:

#     train_df = pd.read_csv(train_after_file_name)
#     x_train, x_valid, y_train, y_valid = train_test_split(train_df.drop(['label_payment_yn'],axis=1), train_df['label_payment_yn'],random_state=21,shuffle=False)
#     catboost_after_model, catboost_pred = catboost_model_fit (x_train, x_valid, y_train, y_valid, 'catboost_after' )  
    
    
    #모델 학습 진행
    for i in range(2):
        if i==0:
            # before 데이터 나누기
            train_df = pd.read_csv(train_before_file_name)
            x_train, x_valid, y_train, y_valid = train_test_split(train_df.drop(['label_payment_yn'],axis=1), train_df['label_payment_yn'],random_state=21,shuffle=False)
            
            #학습시작
            catboost_before_model, catboost_pred = catboost_model_fit (x_train, x_valid, y_train, y_valid, 'catboost_before' )
            before_col = x_train.columns
        else:
            # after 데이터 나누기
            train_df = pd.read_csv(train_after_file_name)
            x_train, x_valid, y_train, y_valid = train_test_split(train_df.drop(['label_payment_yn'],axis=1), train_df['label_payment_yn'], random_state=21,shuffle=False)
            
            #학습 시작
            catboost_after_model, catboost_pred = catboost_model_fit (x_train, x_valid, y_train, y_valid, 'catboost_after' )
            after_col = x_train.columns
        

## 변수 중요도 출력

In [ ]:
description = data_path + "/episdode1_train_description.csv"
description_ob = data_path + "/episdode1_train_ob_description.csv"

description_df=pd.read_csv(description)
description_ob_df=pd.read_csv(description_ob)

#일반 설명과 ob 설명 합치기
description_df = pd.concat([description_df,description_ob_df], ignore_index=True)




for i in range(2):    
    
    #변수 중요도 data  Frame 합치기
    feature_importance_df = pd.DataFrame()
    
    if i ==0:
        feature_importance_dir =before_importance_dir
        feature_importance_df['col'] = before_col
        feature_importance_df['val'] = catboost_before_model.feature_importances_
        
    else:

        feature_importance_dir =after_importance_dir

        feature_importance_df['col'] = after_col
        feature_importance_df['val'] = catboost_after_model.feature_importances_
       



    for row in range(feature_importance_df.shape[0]):
        str_eng= feature_importance_df.loc[row,'col']
        for i in description_df['컬럼 영문명']:
            if i in str_eng:
                feature_importance_df.loc[row,'han'] = description_df.loc[description_df['컬럼 영문명']==i, '컬럼 한글명'].values[0]
                
    #중요 순서로 sort
    feature_importance_df.sort_values(by='val',axis=0,ascending=False,inplace=True)

    #그래프 출력
    #plt.figure(figsize=(25, 5))
    #plt.bar(feature_importance_df['col'],feature_importance_df['val'])
    #plt.xticks(rotation=90)
    #plt.show()

    #파일 출력
    feature_importance_df.to_csv( feature_importance_dir,index=False )


## 모델 결과 비교

In [ ]:

#데이터 csv 불러오기 
before_importance_df = pd.read_csv(before_importance_dir)
after_importance_df = pd.read_csv(after_importance_dir)


#랭크 계산 
before_importance_df['rank']= range(1,before_importance_df.shape[0]+1)
before_importance_df.columns=['col','han','before_score','before_rank']
after_importance_df['rank'] = range(1,after_importance_df.shape[0]+1)
after_importance_df.columns=['col','han','after_score','after_rank']

#이전, 이후 데이터 합치기
compare_importance_df = pd.DataFrame()
compare_importance_df = before_importance_df
compare_importance_df=pd.merge(compare_importance_df,after_importance_df ,how='outer', on='col')
compare_importance_df.drop('han_y',axis=1,inplace=True)


#차이 계산
compare_importance_df['differ_rank']= compare_importance_df['before_rank']-compare_importance_df['after_rank']
compare_importance_df['abs_differ_rank'] = compare_importance_df['differ_rank'].abs()

#정렬
compare_importance_df.sort_values(by='abs_differ_rank',ascending=False ,inplace=True)
compare_importance_df.drop(['abs_differ_rank'],axis=1,inplace=True)

#데이터 출력
compare_importance_df.to_csv(compare_importance_dir, index=False)




In [ ]:
plt.figure(figsize=(15,7))
plt.bar(x=compare_importance_df['col'].head(10),height=compare_importance_df['differ_rank'].head(10))
plt.xticks(compare_importance_df['col'].head(10),fontsize=20,rotation=45)
# compare_importance_df.head(10).plot.bar(x='col', y='differ_rank', rotation=45)
plt.plot()

In [ ]:
#전처리 결과 불러오기 
eda_before_file_name = data_path + "/fit/eda_result_check_"+BEFORE_FILE_NAME+".csv"
eda_after_file_name = data_path + "/fit/eda_result_check_"+AFTER_FILE_NAME+".csv"
compare_importance_df['이전 전처리결과']='-'

#데이터 csv 불러오기 
before_eda_df = pd.read_csv(eda_before_file_name)
after_eda_df = pd.read_csv(eda_after_file_name)



for row in compare_importance_df['col']:
        
    try:
        #이후 데이터 받아오기
        col_str = compare_importance_df.loc[compare_importance_df['col']==row,'col'].values[0]

        for eda_data in before_eda_df['기준컬럼']:
            if eda_data in col_str:
                compare_importance_df.loc[compare_importance_df['col']==row,'이전 전처리결과'] =before_eda_df.loc[before_eda_df['기준컬럼']==eda_data,'전처리결과' ].values[0]
                
       # 이후 전처리 결과 받아오기      
        col_str = compare_importance_df.loc[compare_importance_df['col']==row,'col'].values[0]

        for eda_data in after_eda_df['기준컬럼']:
            if eda_data in col_str:
                
                compare_importance_df.loc[compare_importance_df['col']==row,'이후 전처리결과'] =after_eda_df.loc[after_eda_df['기준컬럼']==eda_data,'전처리결과' ].values[0]
                

    except:
        pass

In [ ]:
pd.set_option('display.max_rows', 500)
compare_importance_df.head(100)